# Environment Setup

In [ ]:
import time
import datetime
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import RobertaForSequenceClassification, RobertaTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, f1_score
import warnings
import psutil

warnings.filterwarnings('ignore')

In [ ]:
# download dataset
import gdown
gdown.download('https://drive.google.com/uc?id=1AQxSOKwt-KavXaXwxMw3h9HyDyuYOYAP', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1AQxSOKwt-KavXaXwxMw3h9HyDyuYOYAP
To: /content/IMDB_Dataset.csv
100%|██████████| 66.2M/66.2M [00:01<00:00, 40.7MB/s]


'IMDB_Dataset.csv'

# BitFit Finetuning

In [ ]:
import time
import datetime
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import RobertaForSequenceClassification, RobertaTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, f1_score
import warnings
import psutil
warnings.filterwarnings('ignore')

def prepare_data(tokenizer, texts, labels=None, max_len=128):
    encoded_batch = tokenizer.batch_encode_plus(
        texts,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoded_batch['input_ids']
    attention_masks = encoded_batch['attention_mask']
    labels = torch.tensor(labels) if labels is not None else None

    return TensorDataset(input_ids, attention_masks, labels)

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load data
train_data = pd.read_csv('IMDB_Dataset.csv')

# Map labels to integers
label_map = {'positive': 1, 'negative': 0}
imdb_reviews = train_data["review"].tolist()
sentiments = [label_map[label] for label in train_data["sentiment"].tolist()]

# Roberta

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

# Initialize tokenizer
model_name = "deepset/tinyroberta-squad2"
#tokenizer = RobertaTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Prepare dataset
train_dataset = prepare_data(tokenizer, imdb_reviews, sentiments)

# Split dataset
train_size = int(0.9 * len(train_dataset))
valid_size = len(train_dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size])

# Create data loaders
num_workers = 4
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=32, num_workers=num_workers)
valid_dataloader = DataLoader(valid_dataset, sampler=SequentialSampler(valid_dataset), batch_size=32, num_workers=num_workers)

# Load RoBERTa
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.to(device)

# Initialize optimizer with bias parameters
bias_params = [p for p in model.parameters() if p.requires_grad]
optimizer = AdamW(bias_params, lr=2e-5, eps=1e-8)

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

# Set up scheduler
epochs = 10
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at deepset/tinyroberta-squad2 and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class BitFitFineTuner(nn.Module):
    def __init__(self, base_model):
        super(BitFitFineTuner, self).__init__()
        self.base_model = base_model
        self.freeze_parameters()

    def freeze_parameters(self):
        for name, param in self.base_model.named_parameters():
            if 'bias' in name:
                param.requires_grad = True
            else:
                param.requires_grad = False

    def forward(self, *input, **kwargs):
        return self.base_model(*input, **kwargs)

In [ ]:
bitfit_model = BitFitFineTuner(model)
bitfit_model.to(device)

# Training loop
for epoch_i in range(0, epochs):
    print('\n======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    t0 = time.time()
    total_train_loss = 0
    model.train()

    # Track memory usage
    mem_before_epoch = psutil.virtual_memory().used / (1024 ** 2)  # Convert to MB for more precision

    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        bitfit_model.zero_grad()
        outputs = bitfit_model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(bitfit_model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)

    mem_after_epoch = psutil.virtual_memory().used / (1024 ** 2)  # Convert to MB for more precision
    print(f"Epoch {epoch_i + 1} Memory used: {mem_after_epoch - mem_before_epoch:.2f} MB")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))

    training_time = format_time(time.time() - t0)
    print("  Training epoch took: {:}".format(training_time))

    # Validation step
    print("Running Validation...")
    t0 = time.time()
    bitfit_model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in valid_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)

        loss = outputs.loss
        logits = outputs.logits
        total_eval_loss += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        nb_eval_steps += 1

    avg_valid_loss = total_eval_loss / len(valid_dataloader)
    avg_val_accuracy = total_eval_accuracy / nb_eval_steps
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_valid_loss))
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    print("  Validation took: {:}".format(validation_time))

print("\nTraining complete!")


======== Epoch 1 / 10 ========
Epoch 1 Memory used: -4.56 MB
  Average training loss: 0.67
  Training epoch took: 0:01:20
Running Validation...
  Validation Loss: 0.61
  Accuracy: 0.73
  Validation took: 0:00:05

======== Epoch 2 / 10 ========
Epoch 2 Memory used: 9.46 MB
  Average training loss: 0.57
  Training epoch took: 0:01:20
Running Validation...
  Validation Loss: 0.51
  Accuracy: 0.78
  Validation took: 0:00:05

======== Epoch 3 / 10 ========
Epoch 3 Memory used: -6.82 MB
  Average training loss: 0.50
  Training epoch took: 0:01:19
Running Validation...
  Validation Loss: 0.48
  Accuracy: 0.80
  Validation took: 0:00:05

======== Epoch 4 / 10 ========
Epoch 4 Memory used: -29.36 MB
  Average training loss: 0.47
  Training epoch took: 0:01:19
Running Validation...
  Validation Loss: 0.45
  Accuracy: 0.81
  Validation took: 0:00:05

======== Epoch 5 / 10 ========
Epoch 5 Memory used: 16.27 MB
  Average training loss: 0.46
  Training epoch took: 0:01:19
Running Validation...
  V

In [ ]:
len([param for name, param in bitfit_model.named_parameters()])

73

# Bert

In [ ]:
# https://huggingface.co/huawei-noah/TinyBERT_General_4L_312D
# TinyBert

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, get_linear_schedule_with_warmup

# Initialize tokenizer
model_name = "huawei-noah/TinyBERT_General_4L_312D"
tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)

# Prepare dataset
train_dataset = prepare_data(tokenizer, imdb_reviews, sentiments)

# Split dataset
train_size = int(0.9 * len(train_dataset))
valid_size = len(train_dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size])

# Create data loaders
num_workers = 4
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=32, num_workers=num_workers)
valid_dataloader = DataLoader(valid_dataset, sampler=SequentialSampler(valid_dataset), batch_size=32, num_workers=num_workers)


# Model and optimizer
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.to(device)

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

# Set up scheduler
epochs = 10
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

pytorch_model.bin:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
bitfit_model = BitFitFineTuner(model)
bitfit_model.to(device)

# Training loop
for epoch_i in range(0, epochs):
    print('\n======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    t0 = time.time()
    total_train_loss = 0
    model.train()

    # Track memory usage
    mem_before_epoch = psutil.virtual_memory().used / (1024 ** 2)  # Convert to MB for more precision

    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        bitfit_model.zero_grad()
        outputs = bitfit_model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(bitfit_model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)

    mem_after_epoch = psutil.virtual_memory().used / (1024 ** 2)  # Convert to MB for more precision
    print(f"Epoch {epoch_i + 1} Memory used: {mem_after_epoch - mem_before_epoch:.2f} MB")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))

    training_time = format_time(time.time() - t0)
    print("  Training epoch took: {:}".format(training_time))

    # Validation step
    print("Running Validation...")
    t0 = time.time()
    bitfit_model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in valid_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)

        loss = outputs.loss
        logits = outputs.logits
        total_eval_loss += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        nb_eval_steps += 1

    avg_valid_loss = total_eval_loss / len(valid_dataloader)
    avg_val_accuracy = total_eval_accuracy / nb_eval_steps
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_valid_loss))
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    print("  Validation took: {:}".format(validation_time))

print("\nTraining complete!")


======== Epoch 1 / 10 ========
Epoch 1 Memory used: -10.21 MB
  Average training loss: 0.69
  Training epoch took: 0:00:24
Running Validation...
  Validation Loss: 0.69
  Accuracy: 0.67
  Validation took: 0:00:01

======== Epoch 2 / 10 ========
Epoch 2 Memory used: 18.06 MB
  Average training loss: 0.69
  Training epoch took: 0:00:24
Running Validation...
  Validation Loss: 0.68
  Accuracy: 0.70
  Validation took: 0:00:02

======== Epoch 3 / 10 ========
Epoch 3 Memory used: 54.68 MB
  Average training loss: 0.68
  Training epoch took: 0:00:24
Running Validation...
  Validation Loss: 0.68
  Accuracy: 0.73
  Validation took: 0:00:01

======== Epoch 4 / 10 ========
Epoch 4 Memory used: -11.82 MB
  Average training loss: 0.68
  Training epoch took: 0:00:24
Running Validation...
  Validation Loss: 0.68
  Accuracy: 0.74
  Validation took: 0:00:01

======== Epoch 5 / 10 ========
Epoch 5 Memory used: -6.10 MB
  Average training loss: 0.68
  Training epoch took: 0:00:24
Running Validation...
 

In [ ]:
len([param for name, param in bitfit_model.named_parameters()])

73